In [1]:
import numpy as np
from multiprocessing import Pool, cpu_count

In [2]:
import time
def timer(func):
    def f(*args, **kwargs):
        start = time.time()
        rv = func(*args, **kwargs)
        duration = time.time() - start
        print("Function '{}' finished after {:.4f} seconds."\
              .format(func.__name__, duration))
        return rv
    return f

In [3]:
class Queue(object):
    def __init__(self):
        self.queue = []
         
    def __call__(self):
        return self.queue
  
    def __len__(self):
        return len(self.queue)
    
    def enqueue(self, item):
        self.queue.append(item)
        
    def dequeue(self):
        return self.queue.pop(0)

    def is_not_empty(self):
        return bool(self.queue)

class Node:
    def __init__(self, state, parent=None, children=None):
        self.state = state
        self.parent = parent
        if self.parent:
            self.depth = parent.depth + 1
        else:
            self.depth = 0
        self.children = children

In [55]:
class ShelfSolver:
    def __init__(self, shelf_size=(5,5), n_shapes=4, n_colors=4, random_seed=None):
        self.shelf_size = shelf_size
        self.n_shapes = n_shapes
        self.n_colors = n_colors
        self.random_seed = random_seed
        self.visited_states = []
    
    def new_random_shelf(self, seeded=False):
        ''' Return a new random shelf matrix '''
        # Create indices
        if seeded: np.random.seed(self.random_seed)
        indices = np.random.choice(
            self.shelf_size[0]*self.shelf_size[1], 
            size = self.n_shapes*self.n_colors,
            replace=False)
        # Create empty shelf
        shelf = np.zeros(self.shelf_size[0]*self.shelf_size[1])
        # Create objects
        objects = np.array([i+j for j in np.arange(10,(self.n_colors+1)*10,10) \
                   for i in np.arange(1,self.n_shapes+1,1)])
        # Populate shelf with objects and reshape
        shelf[indices] = objects
        shelf = shelf.reshape(self.shelf_size)
        shelf = np.asarray(shelf, dtype=int)
        #shelf = np.asarray(shelf, dtype=str)
        return shelf
    
    def assign_bases(self, shelf):
        base_ids = dict()
        for idx, row in enumerate(shelf):
            color_counts = self.count_colors_in_row(row)
            highest_color_counts = np.argwhere(
                color_counts == np.max(color_counts))
            if highest_color_counts.shape[0] == 1:
                base_ids[idx] = highest_color_counts[0,0]+1
            elif highest_color_counts.shape[0] > 1:
                base_ids[idx] = None
                
        # For the leftover colors assign them randomly
        leftover_row_ids = np.argwhere(np.array(list(base_ids.values()))==None)
        leftover_row_ids = [ri[0] for ri in leftover_row_ids] 
        leftover_colors = [color for color in range(1, self.n_colors+1) if color not in list(base_ids.values())]
        assign_ids = np.random.choice(a=leftover_row_ids, size=len(leftover_colors), replace=False)
        for i in range(len(leftover_colors)):
            base_ids[assign_ids[i]] = leftover_colors[i]
    
        return base_ids
    
    def count_colors_in_row(self, row):
        colors_instances = np.zeros(self.n_colors)
        for ob in row:
            if ob != 0:
                color = self.get_color(ob)
                colors_instances[color-1] += 1
        return colors_instances
    
    def most_prominent_color_per_row(self):
        pass
    
    def is_in_correct_base(self, item, base_ids, row_idx):
        return (base_ids[row_idx] == self.get_color(item))
    
    def move_object(self, shelf, obj, old_position, new_position):
        new_shelf = shelf.copy()
        new_shelf[old_position] = 0
        new_shelf[new_position] = obj
        return new_shelf
    
    def is_final_state(self, base_ids, shelf_state):
        is_final_state = np.ones(shelf_state.shape, dtype=bool)
        for row_idx, row in enumerate(shelf_state):
            for col_idx, item in enumerate(row):
                if item != 0:
                    is_final_state[row_idx,col_idx] = \
                       self.is_in_correct_base(item, base_ids, row_idx)
                else:
                    continue
        return is_final_state.all() # all must be True
    
    def has_been_visited(self, shelf_state):
        return hash(str(shelf_state)) in self.visited_states
    
    @timer
    def solve(self, shelf, base_ids, verbose=0):
        
        # Measure elapsed time
        _start = time.time()
        
        # Enqueue start state
        root_node = Node(shelf)
        queue = Queue()
        queue.enqueue(root_node)
        
        # Collect depths of all correct children
        final_states = []
        collected_depths = []
        
        # Iterate through queue
        while queue.is_not_empty():
            
            node = queue.dequeue()
            if self.is_final_state(base_ids, node.state):
                final_states.append(node.state)
                collected_depths.append(node.depth)
                continue
                #return final_states, collected_depths
            
            # Create children of node and enqueue them
            # by iterating through items
            for i in range(self.shelf_size[0]): # iterate thru row
                for j in range(self.shelf_size[1]): # iterate thru col
                    
                    item = node.state[i,j]
                    if verbose: print("\nCurrent item:", item, "at position", (i,j))
                    if item == 0:
                        if verbose: print("Empty space.")
                        continue
                        
                    # If object not in correct base
                    if not self.is_in_correct_base(item, base_ids, i):

                        # Find correct row
                        correct_row = list(base_ids.values())\
                            .index(self.get_color(item))
                        if verbose: print("Correct rowID for item {}: {}".format(item, correct_row))
                        
                        # Find zeros in correct row
                        empty_space_ids = np.argwhere(node.state[correct_row]==0)
                        
                        # Enqueue all states that move item into empty
                        # space in correct row
                        for empty_space_id in empty_space_ids:
                            if verbose: print("Moving item to ({}, {})".format(correct_row,empty_space_id[0]))
                            new_state = self.move_object(
                                node.state, item, (i,j), 
                                (correct_row, empty_space_id))
                            if not self.has_been_visited(new_state):
                                self.visited_states.append(hash(str(new_state)))
                                child_node = Node(new_state, parent=node)
                                if verbose: print(child_node.state)
                                queue.enqueue(child_node)
                        if verbose: print("Length of queue: ", len(queue))
                    else:
                        if verbose: print("Item is in correct base.")
                        continue
            if time.time()-_start > 20:
                print("More than 20 seconds elapsed, terminating with queue length of", len(queue))
                break
        
        return list(np.unique(collected_depths))
            
    def get_color(self, obj):
        return obj//10
    
    def get_shape(self, obj):
        return obj%10, 

In [60]:
solver = ShelfSolver(shelf_size=(4,4), n_colors=3, n_shapes=3, random_seed=80)
solver = ShelfSolver(random_seed=8)
shelf = solver.new_random_shelf()
base_ids = solver.assign_bases(shelf)
print(shelf)
print(base_ids)

[[ 0  0 24  0 21]
 [32 43  0 41 14]
 [12 23 13 11  0]
 [ 0 22  0 33 31]
 [42 34  0  0 44]]
{0: 2, 1: 4, 2: 1, 3: 3, 4: 4}


In [61]:
collected_depths = solver.solve(shelf, base_ids, verbose=0)

Function 'solve' finished after 0.1826 seconds.


In [62]:
collected_depths

[5]

In [63]:
# Collect search depths
depths = []
solver = ShelfSolver()

def thread_job(shelf):
    base_ids = solver.assign_bases(shelf)
    collected_depths = solver.solve(shelf=shelf, base_ids=base_ids)
    return collected_depths

iterable = [solver.new_random_shelf() for _ in range(3)]    
pool = Pool(processes=cpu_count()-1)
collected_depths = pool.map(thread_job, iterable)
depths = [d[0] for d in depths if d]
pool.close()

Function 'solve' finished after 10.2840 seconds.
More than 20 seconds elapsed, terminating with queue length of 11815
More than 20 seconds elapsed, terminating with queue length of 9001
Function 'solve' finished after 20.0088 seconds.
Function 'solve' finished after 20.0098 seconds.


[8]

In [50]:
print(len(depths))
print(depths)

0
[]
